In [1]:
import numpy as np
import nn

# CNNs on MNIST
In the third part of the exercise we will now apply CNNs to MNIST.

First, take a look at the neural network code I placed into the nn package in this repository. It should look familiar as it is mainly the code you used in the last exercise. One thing that I added is a prototyped implementation of convolution and pooling. You will find these in nn/conv/layers.py.

After you have completed exercises 2 a) and 2 b) you should go into that file, and implement the missing pieces, which will essentially be the conv and pool functions you have already written as well as their backward pass (which might be a bit more tricky). 

Once you implemented those, come back here and make sure the following example works.

First, let us do gradient checking using your conv and pooling layers.

In [10]:
input_shape = (5, 1, 28, 28)
n_labels = 6
layers = [nn.InputLayer(input_shape)]

layers.append(nn.Conv(
                layers[-1],
                n_feats=2,
                filter_shape=(3,3),
                init_stddev=0.01,
                activation_fun=nn.Activation('relu'),
))
layers.append(nn.Pool(layers[-1]))
layers.append(nn.Flatten(layers[-1]))
layers.append(nn.FullyConnectedLayer(
                layers[-1],
                num_units=6,
                init_stddev=0.1,
                activation_fun=None
))
layers.append(nn.SoftmaxOutput(layers[-1]))
net = nn.NeuralNetwork(layers)

In [11]:
# create random data
X = np.random.normal(size=input_shape)
Y = np.zeros((input_shape[0], n_labels))
for i in range(Y.shape[0]):
    idx = np.random.randint(n_labels)
    Y[i, idx] = 1.

In [12]:
# perform gradient checking, this should go through if you implemented everything correctly!
net.check_gradients(X, Y)

checking gradient for layer 1
diff 8.93e-01


AssertionError: 

# Train on mnist
Finally, figure out a reasonable network architecture and train it on MNIST.

In [6]:
# you can load the mnist data as 
data = nn.data.mnist()

... downloading MNIST from the web
... loading data
... done loading data
